# Analyzing dask data with SQL

Dask's abilities to analyze data are huge and due to its large similarities with the pandas dataframe API, it is very easy for pandas experts to migrate.
However, for many applications amd users, SQL still plays a large role for querying and retrieving data.
For very good reasons: it is easy to learn, it is a common language many (data) systems understand and it contains all the important elements to query the data.

With [dask-sql](https://nils-braun.github.io/dask-sql/), which leverages [Apache Calcite](https://calcite.apache.org/), it is possible to query the data with SQL and still use the full power of a dask cluster.

`dask-sql` can be installed via conda (or mamba) or pip, like in the following cell:

In [ ]:
! mamba install -y dask-sql


If you want to analyze data with dask-sql in python, you need to do three steps:

1. Create a context
2. Load and register your data 
3. Start querying!


## 1. Create a Context

In SQL, all tables and functions are specified via names. Therefore we need to have some place to store all the registered tables (and functions), so that dask-sql knows which data it refers to.
This is the task of the Context.
You typically create a single context once at the beginning of your python script/notebook and use it through the rest of the application.

In [ ]:
from dask_sql import Context
c = Context()

## Set up a dask cluster

Now would be the best time to connect to your dask cluster if you have one. 
dask-sql leverages dask for performing the computations on the data.

Check out one of [the many ways](https://docs.dask.org/en/latest/setup.html) to create and connect to your dask cluster.

For this example we will create a cluster running locally.
This is optional as dask can also create one implicetly, but we can get more diagnostics and insights.
You can click the link shown after the client intialization to show the dask dashboard.

In [ ]:
from dask.distributed import Client

client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

## 2. Load and register the data

So far, no data was involved. Let's change that! There are many ways how you can get the data into your cluster and tell dask-sql, where to find it. 

### Register a dask or pandas dataframe

If you have already a dask dataframe (which is dask's abstraction of a pandas dataframe with nearly the same API), you can directly associate it with a name:

In [ ]:
from dask.datasets import timeseries
df = timeseries()
type(df)

In [ ]:
c.create_table("timeseries", df.persist())

<div class="alert alert-info">

Please note that we have persisted the data before passing it to dask-sql.
This will tell dask that we want to prefetch the data into memory.
Doing so will speed up the queries a lot, so you probably always want to do this.

</div>

It is also possible to register a pandas dataframe directly.

In [ ]:
import pandas as pd
df = pd.DataFrame({"column": [1, 2, 3]})
c.create_table("pandas", df)

### Read in data an from external location and register it

In most of the cases however, your data will live on some external storage device, such as a local disk, S3 or hdfs.
You can leverage dask's large set of understood input formats and sources to load the data.
Find our more information in the [documentation](https://dask-sql.readthedocs.io/en/latest/pages/data_input.html) of dask-sql.

We have now registered tables in our context with the given names.
Let's do some data analysis!

## 3. Query the data

Whenever you call the `.sql()` method of the context, dask-sql will hand your query to Apache Calcite to turn it into a relational algebra and will then create a dask computation graph, which will execute your computation. 

Let's see how this works in action:

In [ ]:
c.sql("""
    SELECT AVG(x) FROM timeseries
""")

The result is again a dask dataframe, which only represents the computation (without having executed it) so far.
Lets trigger it now!

In [ ]:
c.sql("""
    SELECT AVG(x) FROM timeseries
""").compute()

Congratulations! You have just queried your data with SQL.
If you check out the dask dashboard, you see that this has triggered some computations in dask.

Of course, it is also possible to calculate more complex queries:

In [ ]:
c.sql("""
    SELECT
        lhs.name,
        lhs.id,
        lhs.x
    FROM
        timeseries AS lhs
    JOIN
        (
            SELECT
                name AS max_name,
                MAX(x) AS max_x
            FROM timeseries
            GROUP BY name
        ) AS rhs
    ON
        lhs.name = rhs.max_name AND
        lhs.x = rhs.max_x
""").compute()

You can find more information on dask-sql in the [documentation](https://dask-sql.readthedocs.io/).